In [ ]:
from numpy.fft import fft2, fftshift
import csv 
def radial_profile(data):
    """Computes the radial profile of a 2D FFT result."""
    y, x = np.indices((data.shape))
    center = np.array([y.max() // 2, x.max() // 2])  # Find the center of the FFT image
    
    r = np.sqrt((x - center[1]) ** 2 + (y - center[0]) ** 2)  # Radial distance from the center
    r = r.astype(int)  # Round distances to integers
    
    # Compute the mean value for all points at the same radial distance
    radial_mean = np.bincount(r.ravel(), data.ravel()) / np.bincount(r.ravel())
    
    return radial_mean

def compute_fft_spectrum(img, low_freq_cutoff):
    """Computes the 2D FFT and radial mean profile of the image, with low frequencies removed."""
    # Perform FFT
    fft = np.fft.fft2(img)
    fft_shifted = np.fft.fftshift(fft)
    
    # Compute the magnitude spectrum
    magnitude_spectrum = np.abs(fft_shifted)
    
    # Remove low-frequency bins (around the center)
    center = (magnitude_spectrum.shape[0] // 2, magnitude_spectrum.shape[1] // 2)
    magnitude_spectrum[center[0] - low_freq_cutoff:center[0] + low_freq_cutoff, center[1] - low_freq_cutoff:center[1] + low_freq_cutoff] = 0
    
    # Compute radial profile (amplitude as a function of frequency)
    radial_mean = radial_profile(magnitude_spectrum)
    
    return magnitude_spectrum, radial_mean


def plot_fft_and_radial(spectrum, radial_mean, crop_index):
    """Plots both the 2D FFT magnitude spectrum and the radial profile."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot 2D FFT magnitude spectrum
    ax1.imshow(np.log(1 + spectrum), cmap='gray')
    ax1.set_title(f'2D FFT Magnitude Spectrum (Crop {crop_index + 1})')
    ax1.axis('off')
    
    # Plot radial mean profile
    ax2.plot(radial_mean, label=f'Crop {crop_index + 1}')
    ax2.set_title('Amplitude-Versus-Frequency Spectrum')
    ax2.set_xlabel('Frequency (Radial Distance)')
    ax2.set_ylabel('Amplitude (Mean Magnitude)')
    plt.savefig(f"spectrum_rgb_{crop_index}_BC15-01.png")
    plt.tight_layout()
    plt.show()

def compute_spectra_for_crops(cropped_imgs,lowfrequency_cutoff):
    """Computes the amplitude-versus-frequency spectra for all cropped images."""
    radial_profiles=[]
    for idx, img in enumerate(cropped_imgs):
        if len(img.shape) == 3:  # For RGB images
            # Convert to grayscale by averaging the color channels
            img_gray = np.mean(img, axis=2)
        else:
            img_gray = img  # For grayscale or single-channel images
            
        # Compute the FFT spectrum and radial mean
        magnitude_spectrum, radial_mean = compute_fft_spectrum(img_gray,lowfrequency_cutoff)
        radial_profiles.append(radial_mean)
        # Plot both 2D FFT and radial profile
        plot_fft_and_radial(magnitude_spectrum, radial_mean, idx)
    return radial_profiles

def truncate_profiles_to_min_length(radial_profiles):
    """Truncates all radial profiles to the minimum length found in the data."""
    min_length = min([len(profile) for profile in radial_profiles])  # Find the minimum length
    truncated_profiles = [profile[:min_length] for profile in radial_profiles]  # Truncate each profile
    return np.array(truncated_profiles)

def compute_mean_and_std(radial_profiles):
    """Computes the mean and standard deviation of the truncated radial profiles for all images."""
    # Truncate the profiles to the minimum length
    truncated_profiles = truncate_profiles_to_min_length(radial_profiles)
    
    # Compute the mean and standard deviation across the images for each frequency
    mean_amplitude = np.mean(truncated_profiles, axis=0)
    std_amplitude = np.std(truncated_profiles, axis=0)
    
    return mean_amplitude, std_amplitude

def plot_mean_and_std(mean_amplitude, std_amplitude=None):
    """Plot the mean amplitude with optional standard deviation curves."""
    frequencies = np.arange(len(mean_amplitude))  # Frequency axis (indices)
    
    plt.figure(figsize=(8, 6))
    
    # Plot mean amplitude
    plt.plot(frequencies, mean_amplitude, label='Mean Amplitude', color='blue')
    
    # Optionally, plot mean ± std curves
    if std_amplitude is not None:
        plt.plot(frequencies, mean_amplitude + std_amplitude, '--', color='green', label='Mean + Std')
        plt.plot(frequencies, mean_amplitude - std_amplitude, '--', color='red', label='Mean - Std')
    
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.title('Mean Amplitude Spectrum (with Std. Deviation)')
    plt.legend()
    plt.grid(True)
    plt.show()

def compute_pixel_boundaries(img):
    """Get the maximum boundary value for the image's data type."""
    dtype = img.dtype
    if np.issubdtype(dtype, np.integer):
        return np.iinfo(dtype).max  # Max value for integer types
    elif np.issubdtype(dtype, np.floating):
        return 1.0  # Assume normalized floating-point values are between 0 and 1
    return 1  # Default to 1 if unknown

def normalize_values(values, img):
    """Normalize the values based on the image dtype."""
    boundary = compute_pixel_boundaries(img)
    return values / boundary

def save_max_mean_and_std_csv(species, max_mean, max_std, file_path='Spectrum_all.csv'):
    """Save the max mean and std for the given species in a CSV file."""
    # Open the CSV file in append mode
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header if the file is newly created
        if not file_exists:
            writer.writerow(['Species', 'Max Mean', 'Max Std'])
        
        # Write the species, max_mean, and max_std to the CSV
        writer.writerow([species, max_mean, max_std])
    
    print(f"Max mean and std saved for {species} in {file_path}")

Species=species+'_rgb'
# Assume radial profiles have already been computed
# Compute radial profiles for all RGB and HSI cropped images
radial_profiles_rgb = compute_spectra_for_crops(croped_rgb,5)

# Compute the mean and standard deviation for RGB
mean_rgb, std_rgb = compute_mean_and_std(radial_profiles_rgb)

# Normalize the mean and std based on the dtype of the image
normalized_mean = normalize_values(mean_rgb, croped_rgb[0])
normalized_std = normalize_values(std_rgb, croped_rgb[0])

# Save the max mean and max std values for the species in a CSV file
max_mean_rgb = np.max(normalized_mean)
max_std_rgb = np.max(normalized_std)
save_max_mean_and_std_csv(Species, max_mean_rgb, max_std_rgb)

# Plot the normalized results
plot_mean_and_std(normalized_mean, std_amplitude=normalized_std)

In [ ]:
from numpy.fft import fft2, fftshift
import csv 
def radial_profile(data):
    """Computes the radial profile of a 2D FFT result."""
    y, x = np.indices((data.shape))
    center = np.array([y.max() // 2, x.max() // 2])  # Find the center of the FFT image
    
    r = np.sqrt((x - center[1]) ** 2 + (y - center[0]) ** 2)  # Radial distance from the center
    r = r.astype(int)  # Round distances to integers
    
    # Compute the mean value for all points at the same radial distance
    radial_mean = np.bincount(r.ravel(), data.ravel()) / np.bincount(r.ravel())
    
    return radial_mean

def compute_fft_spectrum(img, low_freq_cutoff):
    """Computes the 2D FFT and radial mean profile of the image, with low frequencies removed."""
    # Perform FFT
    fft = np.fft.fft2(img)
    fft_shifted = np.fft.fftshift(fft)
    
    # Compute the magnitude spectrum
    magnitude_spectrum = np.abs(fft_shifted)
    
    # Remove low-frequency bins (around the center)
    center = (magnitude_spectrum.shape[0] // 2, magnitude_spectrum.shape[1] // 2)
    magnitude_spectrum[center[0] - low_freq_cutoff:center[0] + low_freq_cutoff, center[1] - low_freq_cutoff:center[1] + low_freq_cutoff] = 0
    
    # Compute radial profile (amplitude as a function of frequency)
    radial_mean = radial_profile(magnitude_spectrum)
    
    return magnitude_spectrum, radial_mean


def plot_fft_and_radial(spectrum, radial_mean, crop_index):
    """Plots both the 2D FFT magnitude spectrum and the radial profile."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot 2D FFT magnitude spectrum
    ax1.imshow(np.log(1 + spectrum), cmap='gray')
    ax1.set_title(f'2D FFT Magnitude Spectrum (Crop {crop_index + 1})')
    ax1.axis('off')
    
    # Plot radial mean profile
    ax2.plot(radial_mean, label=f'Crop {crop_index + 1}')
    ax2.set_title('Amplitude-Versus-Frequency Spectrum')
    ax2.set_xlabel('Frequency (Radial Distance)')
    ax2.set_ylabel('Amplitude (Mean Magnitude)')
    plt.savefig(f"spectrum_rgb_{crop_index}_BC15-01.png")
    plt.tight_layout()
    plt.show()

def compute_spectra_for_crops(cropped_imgs,lowfrequency_cutoff):
    """Computes the amplitude-versus-frequency spectra for all cropped images."""
    radial_profiles=[]
    for idx, img in enumerate(cropped_imgs):
        if len(img.shape) == 3:  # For RGB images
            # Convert to grayscale by averaging the color channels
            img_gray = np.mean(img, axis=2)
        else:
            img_gray = img  # For grayscale or single-channel images
            
        # Compute the FFT spectrum and radial mean
        magnitude_spectrum, radial_mean = compute_fft_spectrum(img_gray,lowfrequency_cutoff)
        radial_profiles.append(radial_mean)
        # Plot both 2D FFT and radial profile
        plot_fft_and_radial(magnitude_spectrum, radial_mean, idx)
    return radial_profiles

def truncate_profiles_to_min_length(radial_profiles):
    """Truncates all radial profiles to the minimum length found in the data."""
    min_length = min([len(profile) for profile in radial_profiles])  # Find the minimum length
    truncated_profiles = [profile[:min_length] for profile in radial_profiles]  # Truncate each profile
    return np.array(truncated_profiles)

def compute_mean_and_std(radial_profiles):
    """Computes the mean and standard deviation of the truncated radial profiles for all images."""
    # Truncate the profiles to the minimum length
    truncated_profiles = truncate_profiles_to_min_length(radial_profiles)
    
    # Compute the mean and standard deviation across the images for each frequency
    mean_amplitude = np.mean(truncated_profiles, axis=0)
    std_amplitude = np.std(truncated_profiles, axis=0)
    
    return mean_amplitude, std_amplitude

def plot_mean_and_std(mean_amplitude, std_amplitude=None):
    """Plot the mean amplitude with optional standard deviation curves."""
    frequencies = np.arange(len(mean_amplitude))  # Frequency axis (indices)
    
    plt.figure(figsize=(8, 6))
    
    # Plot mean amplitude
    plt.plot(frequencies, mean_amplitude, label='Mean Amplitude', color='blue')
    
    # Optionally, plot mean ± std curves
    if std_amplitude is not None:
        plt.plot(frequencies, mean_amplitude + std_amplitude, '--', color='green', label='Mean + Std')
        plt.plot(frequencies, mean_amplitude - std_amplitude, '--', color='red', label='Mean - Std')
    
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.title('Mean Amplitude Spectrum (with Std. Deviation)')
    plt.legend()
    plt.grid(True)
    plt.show()

def compute_pixel_boundaries(img):
    """Get the maximum boundary value for the image's data type."""
    dtype = img.dtype
    if np.issubdtype(dtype, np.integer):
        return np.iinfo(dtype).max  # Max value for integer types
    elif np.issubdtype(dtype, np.floating):
        return 1.0  # Assume normalized floating-point values are between 0 and 1
    return 1  # Default to 1 if unknown

def normalize_values(values, img):
    """Normalize the values based on the image dtype."""
    boundary = compute_pixel_boundaries(img)
    return values / boundary

def save_max_mean_and_std_csv(species, max_mean, max_std, file_path='Spectrum_all.csv'):
    """Save the max mean and std for the given species in a CSV file."""
    # Open the CSV file in append mode
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header if the file is newly created
        if not file_exists:
            writer.writerow(['Species', 'Max Mean', 'Max Std'])
        
        # Write the species, max_mean, and max_std to the CSV
        writer.writerow([species, max_mean, max_std])
    
    print(f"Max mean and std saved for {species} in {file_path}")
    
Species=species+'_reconstruct'
# Assume radial profiles have already been computed
# Compute radial profiles for all RGB and HSI cropped images
radial_profiles_rgb = compute_spectra_for_crops(croped_reconstruct,5)

# Compute the mean and standard deviation for RGB
mean_rgb, std_rgb = compute_mean_and_std(radial_profiles_rgb)

# Normalize the mean and std based on the dtype of the image
normalized_mean = normalize_values(mean_rgb, croped_rgb[0])
normalized_std = normalize_values(std_rgb, croped_rgb[0])

# Save the max mean and max std values for the species in a CSV file
max_mean_rgb = np.max(normalized_mean)
max_std_rgb = np.max(normalized_std)
save_max_mean_and_std_csv(Species, max_mean_rgb, max_std_rgb)

# Plot the normalized results
plot_mean_and_std(normalized_mean, std_amplitude=normalized_std)

In [ ]:
Species=species+'_hsi'
# Assume radial profiles have already been computed
# Compute radial profiles for all RGB and HSI cropped images
radial_profiles_rgb = compute_spectra_for_crops(croped_hsi,1)

# Compute the mean and standard deviation for RGB
mean_rgb, std_rgb = compute_mean_and_std(radial_profiles_rgb)

# Normalize the mean and std based on the dtype of the image
normalized_mean = normalize_values(mean_rgb, croped_rgb[0])
normalized_std = normalize_values(std_rgb, croped_rgb[0])

# Save the max mean and max std values for the species in a CSV file
max_mean_rgb = np.max(normalized_mean)
max_std_rgb = np.max(normalized_std)
save_max_mean_and_std_csv(Species, max_mean_rgb, max_std_rgb)

# Plot the normalized results
plot_mean_and_std(normalized_mean, std_amplitude=normalized_std)
                                         

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pyefd import elliptic_fourier_descriptors, normalize_efd, reconstruct_contour
import csv 

def canny_edge_detection(img):
    # Apply Canny edge detector
    edges = cv2.Canny(img, 100, 200)
    return edges

# Apply Canny edge detection to each crop
def apply_canny_to_crops(cropped_images):
    canny_crops = []
    for i, img in enumerate(cropped_images):
        # Convert image to grayscale if it's in color
        img_gray = np.mean(img,axis=2).astype(img.dtype) # For grayscale or single-channel images

        # Apply Canny edge detection
        edges = canny_edge_detection(img_gray)
        
        # Append the edges to the list
        canny_crops.append(edges)

    return canny_crops


def extract_contours(edge_image):
    """Extract contours from the edge-detected image."""
    contours, _ = cv2.findContours(edge_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def compute_efd(contours, order=10):
    """Compute the Elliptic Fourier Descriptors for each contour."""
    efd_coeffs = []
    for contour in contours:
        contour = np.squeeze(contour)
        if len(contour) > 10:  # Ensure contour has enough points
            coeffs = elliptic_fourier_descriptors(contour, order=order, normalize=False)
            efd_coeffs.append(coeffs)
    return efd_coeffs

def rescale_contour(contour, original_image_shape):
    """Rescale the reconstructed contour to the original image dimensions."""
    height, width = original_image_shape[:2]
    
    # Normalize contour between 0 and 1 (as EFD often works with normalized coordinates)
    contour_min = np.min(contour, axis=0)
    contour_max = np.max(contour, axis=0)
    
    normalized_contour = (contour - contour_min) / (contour_max - contour_min)
    
    # Scale the normalized contour to the size of the original image
    rescaled_contour = np.zeros_like(normalized_contour)
    rescaled_contour[:, 0] = normalized_contour[:, 0] * width  # Scale x coordinates
    rescaled_contour[:, 1] = normalized_contour[:, 1] * height  # Scale y coordinates
    
    return rescaled_contour

def reconstruct_shape(coeffs, original_image_shape):
    """Reconstruct the shape from normalized EFD coefficients."""
    # Normalize EFD coefficients
    coeffs = normalize_efd(coeffs)
    # Reconstruct contour from normalized EFD
    reconstructed_contour = reconstruct_contour(coeffs, locus=(0, 0), num_points=500)
    
    # Rescale the reconstructed contour to the original image dimensions
    rescaled_contour = rescale_contour(reconstructed_contour, original_image_shape)
    
    return rescaled_contour

def plot_contours(original_img, original_contours, reconstructed_contours, contour_features, image_index):
    """Plot the original contours and the reconstructed ones side by side, 
    with morphological features marked on the reconstructed contours."""
    plt.figure(figsize=(12, 6))

    # Plot original contours
    plt.subplot(1, 2, 1)
    plt.imshow(original_img, cmap='gray')
    for contour in original_contours:
        contour = np.squeeze(contour)
        plt.plot(contour[:, 0], contour[:, 1], color='blue')
    plt.title("Original Contours")

    # Plot reconstructed contours with features
    plt.subplot(1, 2, 2)
    plt.imshow(np.ones((200,60)) * 255, cmap='gray')  # White background
    
    for idx, (contour, features) in enumerate(zip(reconstructed_contours, contour_features)):
        plt.plot(contour[:, 0], contour[:, 1], color='red')
        
        # Calculate the center of the contour
        center_x = np.mean(contour[:, 0])
        center_y = np.mean(contour[:, 1])
        
        # Mark major and minor axis lengths
        plt.text(center_x, center_y, f"f2: {features['f2']:.2f}\nf3: {features['f3']:.2f}", 
                ha='center', va='center', fontsize=8, color='green')
        
    plt.title("Reconstructed Contours from EFD")

    plt.tight_layout()
    plt.show()

def compute_morphological_features(contour):
    """Compute morphological features of the given contour."""
    if len(contour.shape) == 2:
        contour = np.expand_dims(contour, axis=1)
    contour = np.array(contour, dtype=np.float32)
    features = {}

    # Feature f1: Area (number of pixels inside the contour)
    f1_area = cv2.contourArea(contour)
    
    # Feature f5: Perimeter (number of pixels along the boundary)
    f5_perimeter = cv2.arcLength(contour, True)
    
    # Check if the contour has enough points to fit an ellipse
    if len(contour) >= 5:
        # Fit an ellipse to the contour
        ellipse = cv2.fitEllipse(contour)
        
        # Get major and minor axis lengths (f2 and f3)
        (center, (major_axis_length, minor_axis_length), angle) = ellipse
        
        f2_major_axis_length = max(major_axis_length, minor_axis_length)  # Major axis
        f3_minor_axis_length = min(major_axis_length, minor_axis_length)  # Minor axis
        
        # Feature f4: MinorAxisLength / MajorAxisLength
        f4_axis_ratio = f3_minor_axis_length / f2_major_axis_length
        
        # Feature f6: FociDistance / MajorAxisLength (Eccentricity)
        foci_distance = np.sqrt(f2_major_axis_length**2 - f3_minor_axis_length**2)
        f6_eccentricity = foci_distance / f2_major_axis_length
    else:
        # If there are not enough points to fit an ellipse, set features to zero or NaN
        f2_major_axis_length = f3_minor_axis_length = f4_axis_ratio = f6_eccentricity = np.nan
    
    # Feature f5: Perimeter / Area
    f5_perimeter_area_ratio = f5_perimeter / f1_area if f1_area != 0 else np.nan
    
    # Store features in a dictionary
    features['f1'] = f1_area
    features['f2'] = f2_major_axis_length
    features['f3'] = f3_minor_axis_length
    features['f4'] = f4_axis_ratio
    features['f5'] = f5_perimeter_area_ratio
    features['f6'] = f6_eccentricity
    
    return features
def apply_closing(canny_image, kernel_size=(3, 3)):
    """
    Apply morphological closing on the given image.
    
    Parameters:
    canny_image (np.ndarray): The input edge-detected image (e.g., from Canny edge detection).
    kernel_size (tuple): The size of the structuring element (kernel) used for closing.

    Returns:
    np.ndarray: The image after applying morphological closing.
    """
    image=canny_image
    for i in range(5):
        # Create a structuring element (kernel) for closing
        kernel = np.ones(kernel_size, np.uint8)
        
        # Apply morphological closing (dilation followed by erosion)
        image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    closed_image = image
    return closed_image
def compute_features_for_contours(contours):
    """Compute morphological features for each contour."""
    all_features = []
    for contour in contours:
        features = compute_morphological_features(contour)
        all_features.append(features)
    return all_features

def remove_small_contours(image, min_area=80):
    """
    Remove small contours from the image based on a minimum area threshold.
    
    Parameters:
    image (np.ndarray): The input binary image (typically an edge-detected or thresholded image).
    min_area (int): The minimum area threshold for keeping contours. Contours with area smaller than this will be removed.

    Returns:
    np.ndarray: The image with small contours removed.
    """
    # Find all contours in the image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create an empty mask to draw the filtered contours
    mask = np.zeros_like(image)
    
    # Loop over each contour
    for contour in contours:
        # Calculate the contour area
        area = cv2.contourArea(contour)
        
        # If the contour area is larger than the minimum area, draw it on the mask
        if area >= min_area:
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)  # Fill the contour
    
    return mask
def draw_contours_on_image(image, contours):
    """
    Draw contours on the original image and display them.

    Parameters:
    image (np.ndarray): The original image or edge-detected image (e.g., Canny image).
    contours (list): List of contours to draw on the image.

    Returns:
    None: Displays the image with contours using matplotlib.
    """
    # Create a copy of the original image to draw on
    image_with_contours = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert grayscale image to BGR for colored contours
    
    # Draw the contours on the image (green color with thickness of 2)
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)
    
    # Display the result using matplotlib
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(image_with_contours, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct color display in matplotlib
    plt.title('Contours Overlayed on the Image')
    plt.axis('off')
    plt.show()
def clean_image(canny_image):
    # Step 1: Apply morphological closing with a kernel size of (1, 1)
    closed_canny_image = apply_closing(canny_image, kernel_size=(3, 3))
    # Step 2: Remove small contours based on area
    cleaned_canny_image = remove_small_contours(closed_canny_image, min_area=80)
    return cleaned_canny_image
def compute_mean_and_std_for_all_crops(all_features):
    """Compute the mean and std for each feature (f1 to f6) across all crops."""
    features_array = {f'f{i}': [] for i in range(1, 7)}
    
    for features in all_features:
        for key in features_array.keys():
            features_array[key].append(features[key])

    mean_features = {key: np.nanmean(values) for key, values in features_array.items()}
    std_features = {key: np.nanstd(values) for key, values in features_array.items()}

    return mean_features, std_features
def save_mean_std_to_csv(species, mean_features, std_features, file_path='Morphological_all.csv'):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Species', 'Mean_f1', 'Mean_f2', 'Mean_f3', 'Mean_f4', 'Mean_f5', 'Mean_f6',
                             'Std_f1', 'Std_f2', 'Std_f3', 'Std_f4', 'Std_f5', 'Std_f6'])
        
        mean_row = [mean_features[f'f{i}'] for i in range(1, 7)]
        std_row = [std_features[f'f{i}'] for i in range(1, 7)]
        
        writer.writerow([species] + mean_row + std_row)
        
def is_closed_contour(contour, tolerance=3):
    """Check if the contour is 'closed' by comparing the start and end points."""
    start_point = contour[0]
    end_point = contour[-1]
    distance = np.linalg.norm(start_point - end_point)
    return distance < tolerance
    
Species=species+'_rgb'
# Apply Canny edge detection to RGB and HSI crops
canny_rgb_crops = apply_canny_to_crops(croped_rgb)
all_contour_features = []
for i in range(48):
    # Assuming canny_rgb_crops[6] contains your edge-detected image
    canny_image = clean_image(canny_rgb_crops[i])
    
    # Step 1: Extract contours from the edge-detected image
    contours = extract_contours(canny_image)
    # Filter contours that are not closed
    closed_contours = [contour for contour in contours if is_closed_contour(contour)]
    
    if not closed_contours:
        continue
    
    # Visualize contours on the Canny image to check if they are correct
    draw_contours_on_image(canny_image, contours)
    
    # Step 2: Compute EFD coefficients for each contour
    efd_coeffs = compute_efd(contours)
    
    # Step 3: Reconstruct the shape using normalized EFD and rescale it to original image size
    reconstructed_contours = [reconstruct_shape(coeffs, (200, 60)) for coeffs in efd_coeffs]
    # Assuming reconstructed_contours contains the reconstructed contours
    contour_features = compute_features_for_contours(reconstructed_contours)
    all_contour_features.extend(contour_features)
    # Display the computed features for each contour    
    # Step 4: Plot original and reconstructed contours
    print(i)
    plot_contours(canny_image, contours, reconstructed_contours, contour_features,i)

# Compute mean and standard deviation of features across all crops
mean_features, std_features = compute_mean_and_std_for_all_crops(all_contour_features)
# Save mean and std features to CSV
save_mean_std_to_csv(Species, mean_features, std_features)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pyefd import elliptic_fourier_descriptors, normalize_efd, reconstruct_contour
import csv 

def canny_edge_detection(img):
    # Apply Canny edge detector
    edges = cv2.Canny(img, 100, 200)
    return edges

# Apply Canny edge detection to each crop
def apply_canny_to_crops(cropped_images):
    canny_crops = []
    for i, img in enumerate(cropped_images):
        # Convert image to grayscale if it's in color
        img_gray = np.mean(img,axis=2).astype(img.dtype) # For grayscale or single-channel images

        # Apply Canny edge detection
        edges = canny_edge_detection(img_gray)
        
        # Append the edges to the list
        canny_crops.append(edges)

    return canny_crops


def extract_contours(edge_image):
    """Extract contours from the edge-detected image."""
    contours, _ = cv2.findContours(edge_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def compute_efd(contours, order=10):
    """Compute the Elliptic Fourier Descriptors for each contour."""
    efd_coeffs = []
    for contour in contours:
        contour = np.squeeze(contour)
        if len(contour) > 10:  # Ensure contour has enough points
            coeffs = elliptic_fourier_descriptors(contour, order=order, normalize=False)
            efd_coeffs.append(coeffs)
    return efd_coeffs

def rescale_contour(contour, original_image_shape):
    """Rescale the reconstructed contour to the original image dimensions."""
    height, width = original_image_shape[:2]
    
    # Normalize contour between 0 and 1 (as EFD often works with normalized coordinates)
    contour_min = np.min(contour, axis=0)
    contour_max = np.max(contour, axis=0)
    
    normalized_contour = (contour - contour_min) / (contour_max - contour_min)
    
    # Scale the normalized contour to the size of the original image
    rescaled_contour = np.zeros_like(normalized_contour)
    rescaled_contour[:, 0] = normalized_contour[:, 0] * width  # Scale x coordinates
    rescaled_contour[:, 1] = normalized_contour[:, 1] * height  # Scale y coordinates
    
    return rescaled_contour

def reconstruct_shape(coeffs, original_image_shape):
    """Reconstruct the shape from normalized EFD coefficients."""
    # Normalize EFD coefficients
    coeffs = normalize_efd(coeffs)
    # Reconstruct contour from normalized EFD
    reconstructed_contour = reconstruct_contour(coeffs, locus=(0, 0), num_points=500)
    
    # Rescale the reconstructed contour to the original image dimensions
    rescaled_contour = rescale_contour(reconstructed_contour, original_image_shape)
    
    return rescaled_contour

def plot_contours(original_img, original_contours, reconstructed_contours, contour_features, image_index):
    """Plot the original contours and the reconstructed ones side by side, 
    with morphological features marked on the reconstructed contours."""
    plt.figure(figsize=(12, 6))

    # Plot original contours
    plt.subplot(1, 2, 1)
    plt.imshow(original_img, cmap='gray')
    for contour in original_contours:
        contour = np.squeeze(contour)
        plt.plot(contour[:, 0], contour[:, 1], color='blue')
    plt.title("Original Contours")

    # Plot reconstructed contours with features
    plt.subplot(1, 2, 2)
    plt.imshow(np.ones((200,60)) * 255, cmap='gray')  # White background
    
    for idx, (contour, features) in enumerate(zip(reconstructed_contours, contour_features)):
        plt.plot(contour[:, 0], contour[:, 1], color='red')
        
        # Calculate the center of the contour
        center_x = np.mean(contour[:, 0])
        center_y = np.mean(contour[:, 1])
        
        # Mark major and minor axis lengths
        plt.text(center_x, center_y, f"f2: {features['f2']:.2f}\nf3: {features['f3']:.2f}", 
                ha='center', va='center', fontsize=8, color='green')
        
    plt.title("Reconstructed Contours from EFD")

    plt.tight_layout()
    plt.show()

def compute_morphological_features(contour):
    """Compute morphological features of the given contour."""
    if len(contour.shape) == 2:
        contour = np.expand_dims(contour, axis=1)
    contour = np.array(contour, dtype=np.float32)
    features = {}

    # Feature f1: Area (number of pixels inside the contour)
    f1_area = cv2.contourArea(contour)
    
    # Feature f5: Perimeter (number of pixels along the boundary)
    f5_perimeter = cv2.arcLength(contour, True)
    
    # Check if the contour has enough points to fit an ellipse
    if len(contour) >= 5:
        # Fit an ellipse to the contour
        ellipse = cv2.fitEllipse(contour)
        
        # Get major and minor axis lengths (f2 and f3)
        (center, (major_axis_length, minor_axis_length), angle) = ellipse
        
        f2_major_axis_length = max(major_axis_length, minor_axis_length)  # Major axis
        f3_minor_axis_length = min(major_axis_length, minor_axis_length)  # Minor axis
        
        # Feature f4: MinorAxisLength / MajorAxisLength
        f4_axis_ratio = f3_minor_axis_length / f2_major_axis_length
        
        # Feature f6: FociDistance / MajorAxisLength (Eccentricity)
        foci_distance = np.sqrt(f2_major_axis_length**2 - f3_minor_axis_length**2)
        f6_eccentricity = foci_distance / f2_major_axis_length
    else:
        # If there are not enough points to fit an ellipse, set features to zero or NaN
        f2_major_axis_length = f3_minor_axis_length = f4_axis_ratio = f6_eccentricity = np.nan
    
    # Feature f5: Perimeter / Area
    f5_perimeter_area_ratio = f5_perimeter / f1_area if f1_area != 0 else np.nan
    
    # Store features in a dictionary
    features['f1'] = f1_area
    features['f2'] = f2_major_axis_length
    features['f3'] = f3_minor_axis_length
    features['f4'] = f4_axis_ratio
    features['f5'] = f5_perimeter_area_ratio
    features['f6'] = f6_eccentricity
    
    return features
def apply_closing(canny_image, kernel_size=(3, 3)):
    """
    Apply morphological closing on the given image.
    
    Parameters:
    canny_image (np.ndarray): The input edge-detected image (e.g., from Canny edge detection).
    kernel_size (tuple): The size of the structuring element (kernel) used for closing.

    Returns:
    np.ndarray: The image after applying morphological closing.
    """
    image=canny_image
    for i in range(5):
        # Create a structuring element (kernel) for closing
        kernel = np.ones(kernel_size, np.uint8)
        
        # Apply morphological closing (dilation followed by erosion)
        image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    closed_image = image
    return closed_image
def compute_features_for_contours(contours):
    """Compute morphological features for each contour."""
    all_features = []
    for contour in contours:
        features = compute_morphological_features(contour)
        all_features.append(features)
    return all_features

def remove_small_contours(image, min_area=80):
    """
    Remove small contours from the image based on a minimum area threshold.
    
    Parameters:
    image (np.ndarray): The input binary image (typically an edge-detected or thresholded image).
    min_area (int): The minimum area threshold for keeping contours. Contours with area smaller than this will be removed.

    Returns:
    np.ndarray: The image with small contours removed.
    """
    # Find all contours in the image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create an empty mask to draw the filtered contours
    mask = np.zeros_like(image)
    
    # Loop over each contour
    for contour in contours:
        # Calculate the contour area
        area = cv2.contourArea(contour)
        
        # If the contour area is larger than the minimum area, draw it on the mask
        if area >= min_area:
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)  # Fill the contour
    
    return mask
def draw_contours_on_image(image, contours):
    """
    Draw contours on the original image and display them.

    Parameters:
    image (np.ndarray): The original image or edge-detected image (e.g., Canny image).
    contours (list): List of contours to draw on the image.

    Returns:
    None: Displays the image with contours using matplotlib.
    """
    # Create a copy of the original image to draw on
    image_with_contours = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert grayscale image to BGR for colored contours
    
    # Draw the contours on the image (green color with thickness of 2)
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)
    
    # Display the result using matplotlib
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(image_with_contours, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct color display in matplotlib
    plt.title('Contours Overlayed on the Image')
    plt.axis('off')
    plt.show()
def clean_image(canny_image):
    # Step 1: Apply morphological closing with a kernel size of (1, 1)
    closed_canny_image = apply_closing(canny_image, kernel_size=(3, 3))
    # Step 2: Remove small contours based on area
    cleaned_canny_image = remove_small_contours(closed_canny_image, min_area=80)
    return cleaned_canny_image
def compute_mean_and_std_for_all_crops(all_features):
    """Compute the mean and std for each feature (f1 to f6) across all crops."""
    features_array = {f'f{i}': [] for i in range(1, 7)}
    
    for features in all_features:
        for key in features_array.keys():
            features_array[key].append(features[key])

    mean_features = {key: np.nanmean(values) for key, values in features_array.items()}
    std_features = {key: np.nanstd(values) for key, values in features_array.items()}

    return mean_features, std_features
def save_mean_std_to_csv(species, mean_features, std_features, file_path='Morphological_all.csv'):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Species', 'Mean_f1', 'Mean_f2', 'Mean_f3', 'Mean_f4', 'Mean_f5', 'Mean_f6',
                             'Std_f1', 'Std_f2', 'Std_f3', 'Std_f4', 'Std_f5', 'Std_f6'])
        
        mean_row = [mean_features[f'f{i}'] for i in range(1, 7)]
        std_row = [std_features[f'f{i}'] for i in range(1, 7)]
        
        writer.writerow([species] + mean_row + std_row)
        
def is_closed_contour(contour, tolerance=3):
    """Check if the contour is 'closed' by comparing the start and end points."""
    start_point = contour[0]
    end_point = contour[-1]
    distance = np.linalg.norm(start_point - end_point)
    return distance < tolerance
Species=species+'_reconstruct'

def get_reference_values(species_csv, species):
    """Get the reference mean and std values for f2 and f3 from the CSV."""
    with open(species_csv, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Species'] == species:
                mean_f2 = float(row['Mean_f2'])
                std_f2 = float(row['Std_f2'])
                mean_f3 = float(row['Mean_f3'])
                std_f3 = float(row['Std_f3'])
                return mean_f2, std_f2, mean_f3, std_f3
    return None

def is_within_valid_range(features, mean_f2, std_f2, mean_f3, std_f3):
    """Check if f2 and f3 of the current features are within valid ranges."""
    f2 = features['f2']
    f3 = features['f3']
    
    valid_f2_range = [mean_f2 - 3*std_f2, mean_f2 + 3*std_f2]
    valid_f3_range = [mean_f3 - 3*std_f3, mean_f3 + 3*std_f3]
    
    return (valid_f2_range[0] <= f2 <= valid_f2_range[1]) and (valid_f3_range[0] <= f3 <= valid_f3_range[1])


# Example: Get reference values for species 'species_rgb'
species_csv = 'Morphological_all.csv'
species_ref = species+'_rgb'
mean_f2_ref, std_f2_ref, mean_f3_ref, std_f3_ref = get_reference_values(species_csv, species_ref)

# Apply Canny edge detection to RGB and HSI crops
def Extract_rgb_bands(croped_reconstructed):
    croped_reconstruct_rgb = [crops[:,:,rgb_bands] for crops in croped_reconstructed]
    return croped_reconstruct_rgb   
    
croped_reconstruct_rgb= Extract_rgb_bands(croped_reconstruct)
croped_reconstruct_rgb_uint8 = convert_list_to_uint8(croped_reconstruct_rgb)   
canny_rgb_crops = apply_canny_to_crops(croped_reconstruct_rgb_uint8)
valid_contour_features = []

for i in range(48):
    canny_image = clean_image(canny_rgb_crops[i])
    
    # Extract and filter contours
    contours = extract_contours(canny_image)
    closed_contours = [contour for contour in contours if is_closed_contour(contour)]
    
    if not closed_contours:
        continue
    
    # Compute EFD and reconstruct shapes
    efd_coeffs = compute_efd(closed_contours)
    reconstructed_contours = [reconstruct_shape(coeffs, (200, 60)) for coeffs in efd_coeffs]
    
    # Compute contour features
    contour_features = compute_features_for_contours(reconstructed_contours)
    
    # Check validity of the features (f2 and f3)
    for features in contour_features:
        if is_within_valid_range(features, mean_f2_ref, std_f2_ref, mean_f3_ref, std_f3_ref):
            valid_contour_features.append(features)
    
    # Plot contours if valid
    if valid_contour_features:
        plot_contours(canny_image, contours, reconstructed_contours, valid_contour_features, i)
    print(f"Processed crop {i}")

# Compute mean and std for the valid features across all crops
mean_features, std_features = compute_mean_and_std_for_all_crops(valid_contour_features)

# Save the valid results to CSV
save_mean_std_to_csv(Species, mean_features, std_features)

In [ ]:
import pandas as pd
def compute_wavelength_profile(cropped_hsi, cropped_thresh):
    """Compute the mean intensity across spectral bands for each crop, excluding background."""
    wavelength_profiles = []
    
    # Loop over each cropped region
    for crop_hsi, crop_thresh in zip(cropped_hsi, cropped_thresh):
        mean_intensity_per_band = []
        
        # For each spectral band in the HSI image (assumed to be along axis 2)
        num_bands = crop_hsi.shape[2]
        for band in range(num_bands):
            # Extract the band and corresponding threshold mask
            band_image = crop_hsi[:, :, band]
            thresh_mask = crop_thresh != 255  # Mask where the pixels are not background
            
            # Compute mean intensity of non-background pixels
            mean_intensity = np.mean(band_image[thresh_mask])
            mean_intensity_per_band.append(mean_intensity)
        
        # Store the intensity profile for the current crop
        wavelength_profiles.append(mean_intensity_per_band)
    
    return wavelength_profiles

def plot_wavelength_profiles(wavelength_profiles, num_crops):
    """Plot the wavelength profile for each cropped image."""
    for i, profile in enumerate(wavelength_profiles):
        plt.plot(profile, label=f'Crop {i + 1}')
    
    plt.xlabel('Band Index')
    plt.ylabel('Mean Intensity (Excluding Background)')
    plt.title(f'Wavelength Profiles for {num_crops} Crops')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.show()

def save_mean_std_wavelength_profiles_to_csv(wavelength_profiles, species, file_name='wavprofile.csv'):
    """Compute and save the mean and std of the wavelength profiles across all crops to a CSV file."""
    # Convert the list of wavelength profiles to a NumPy array
    wavelength_profiles = np.array(wavelength_profiles)
    
    # Compute mean and std across all crops for each band (axis 0 is the crop dimension)
    mean_profile = np.mean(wavelength_profiles, axis=0)
    std_profile = np.std(wavelength_profiles, axis=0)
    
    # Create a dictionary for the data to save with 160 columns (80 for mean, 80 for std)
    data = {'Species': [species]}
    
    # Add mean and std columns for each band (Band_100 to Band_179)
    for band in range(100, 180):
        data[f'mean_{band}'] = [mean_profile[band]]
        data[f'std_{band}'] = [std_profile[band]]
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)
    
    # Check if the file already exists
    file_exists = os.path.isfile(file_name)
    
    if not file_exists:
        # If file doesn't exist, create a header row with 'Species', 'mean_100', 'std_100', ..., 'mean_179', 'std_179'
        columns = ['Species']
        for band in range(100, 180):
            columns.append(f'mean_{band}')
            columns.append(f'std_{band}')
        
        # Create a DataFrame with just the column names and write to CSV
        header_df = pd.DataFrame(columns=[columns])
        header_df.to_csv(file_name, mode='w', index=False, header=True)
    
    # Save the DataFrame to CSV (append if file exists, otherwise create)
    df.to_csv(file_name, mode='a', index=False, header=False) 
    
Species=species+'_hsi'

# Assuming `croped_hsi` is the cropped HSI images and `croped_thresh_hsi` is the corresponding thresholded images
wavelength_profiles_hsi = compute_wavelength_profile(croped_hsi, croped_thresh_hsi)

# Plot the wavelength profiles for the HSI crops (48 curves)
plot_wavelength_profiles(wavelength_profiles_hsi, num_crops=len(wavelength_profiles_hsi))

save_mean_std_wavelength_profiles_to_csv(wavelength_profiles_hsi, Species)

In [ ]:
Species=species+'_reconstruct'

# Assuming `croped_hsi` is the cropped HSI images and `croped_thresh_hsi` is the corresponding thresholded images
wavelength_profiles_hsi = compute_wavelength_profile(croped_reconstruct, croped_thresh_reconstruct)

# Plot the wavelength profiles for the HSI crops (48 curves)
plot_wavelength_profiles(wavelength_profiles_hsi, num_crops=len(wavelength_profiles_hsi))

save_mean_std_wavelength_profiles_to_csv(wavelength_profiles_hsi, Species)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def compute_pixel_wavelength_profiles(cropped_hsi, cropped_thresh):
    """Compute the intensity values across spectral bands for each non-background pixel."""
    pixel_wavelength_profiles = []
    
    # Loop over each cropped region
    for crop_hsi, crop_thresh in zip(cropped_hsi, cropped_thresh):
        profiles_for_crop = []
        
        # Get non-background pixel positions (where the threshold mask is not 255)
        non_background_indices = np.where(crop_thresh != 255)
        
        # For each non-background pixel, extract its intensity across all bands
        for y, x in zip(*non_background_indices):
            # Extract the pixel's intensity across all bands
            pixel_profile = crop_hsi[y, x, :]  # This will be a vector of length equal to the number of bands
            profiles_for_crop.append(pixel_profile)
        
        pixel_wavelength_profiles.append(profiles_for_crop)
    
    return pixel_wavelength_profiles

def plot_pixel_wavelength_profiles(wavelength_profiles, crop_idx):
    """Plot wavelength profiles for each non-background pixel in a crop with mean and std."""
    plt.figure(figsize=(16, 6))
    
    # Get the number of profiles and create a color map
    num_profiles = len(wavelength_profiles)
    colors = cm.get_cmap('rainbow', num_profiles)  # Use the 'rainbow' colormap for distinct colors
    
    # Plot each pixel's wavelength profile with different colors
    plt.subplot(1, 2, 1)
    for i, profile in enumerate(wavelength_profiles):
        plt.plot(profile, color=colors(i), alpha=0.7)  # Assign different colors to each profile
    
    plt.xlabel('Band Index')
    plt.ylabel('Intensity')
    plt.title(f'Wavelength Profiles for Crop {crop_idx + 1}')
    plt.grid(True)
    
    # Convert wavelength_profiles to a numpy array for mean/std calculations
    wavelength_profiles = np.array(wavelength_profiles)
    
    # Compute the mean and standard deviation across all pixels in the crop
    mean_profile = np.mean(wavelength_profiles, axis=0)
    std_profile = np.std(wavelength_profiles, axis=0)
    
    # Plot the mean and std in a separate subplot
    plt.subplot(1, 2, 2)
    plt.plot(mean_profile, label='Mean', color='blue', linewidth=2)
    plt.fill_between(range(len(mean_profile)), 
                     mean_profile - std_profile, 
                     mean_profile + std_profile, 
                     color='blue', alpha=0.3, label='± 1 Std. Dev.')
    
    plt.xlabel('Band Index')
    plt.ylabel('Intensity')
    plt.title(f'Mean and Std. Dev. for Crop {crop_idx + 1}')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Compute pixel-wise wavelength profiles for all HSI cropped images
pixel_wavelength_profiles_hsi = compute_pixel_wavelength_profiles(croped_hsi, croped_thresh_hsi)

# Plot the wavelength profiles and mean/std for each crop
for crop_idx, profiles in enumerate(pixel_wavelength_profiles_hsi):
    plot_pixel_wavelength_profiles(profiles, crop_idx)
